# Deploy webservice

In [ ]:
from azureml.core.model import Model, InferenceConfig
from azureml.core import Workspace, Environment 
from azureml.core.webservice import AciWebservice

ws = Workspace.from_config()
model = Model(ws, 'german-credit-basic-model')

In [ ]:
# Define deployment setup

# Conda enviroment (if we want to use additional Python packages)
env = Environment.from_conda_specification("inference-env", "../config/inference-conda.yml")

inference_config = InferenceConfig(entry_script='../score.py', environment=env)

# Define how our webservice should look like (resources, security, etc.)
deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

In [ ]:
service = Model.deploy(ws, "model-test-deployment", [model], inference_config, deployment_config)
service.wait_for_deployment(show_output=True)

In [ ]:
service.scoring_uri

In [ ]:
import requests
import json

url = service.scoring_uri

test_data = {
  'data': [{
    "Age": 20,
    "Sex": "male",
    "Job": 0,
    "Housing": "own",
    "Saving accounts": "little",
    "Checking account": "little",
    "Credit amount": 100,
    "Duration": 48,
    "Purpose": "radio/TV"
  }]
}

headers = {'Content-Type':'application/json'}
resp = requests.post(url, json=test_data, headers=headers)

print("Prediction (good, bad):", resp.text)